Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "10")
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_02_model = "
data{
    int N;
    int pulled_left[N];
    int prosoc_left[N];
}
parameters{
    real a;
    real bp;
}
model{
    vector[N] p;
    bp ~ normal( 0 , 10 );
    a ~ normal( 0 , 10 );
    for ( i in 1:N ) {
        p[i] = a + bp * prosoc_left[i];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [5]:
stanmodel = Stanmodel(name="m_10_02_model",
monitors = ["a", "bp"],
model=m_10_02_model, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/10/tmp/m_10_02_model.stan will be updated.



Input data for cmdstan

In [6]:
m_10_02_data = Dict("N" => size(df, 1),
"pulled_left" => df[:pulled_left], "prosoc_left" => df[:prosoc_left]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_02_data, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
# Result rethinking
rethinking = "
   mean   sd  5.5% 94.5% n_eff Rhat
a  0.04 0.12 -0.16  0.21   180 1.00
bp 0.57 0.19  0.30  0.87   183 1.01
";

Describe the draws

In [8]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
       Mean        SD       Naive SE       MCSE      ESS
 a 0.049066894 0.12450227 0.0019685538 0.0034323052 1000
bp 0.560999904 0.18026383 0.0028502214 0.0047392469 1000

Quantiles:
       2.5%       25.0%     50.0%     75.0%      97.5%  
 a -0.19593617 -0.0315014 0.0447517 0.1354885 0.29192613
bp  0.20368850  0.4386055 0.5602940 0.6854640 0.90926998



End of `10/m10.02s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*